In [3]:
#import mysql
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

## Downloand database

In [232]:
# def connect_to_database():
#     try:
#         with open('secret.txt', 'r') as file:
#             connection = mysql.connector.connect(
#                 host='localhost',
#                 database='magic_cards',
#                 user='root',
#                 password=file.readline()
#             )
#         if connection.is_connected():
#             print("Connected to MySQL database")
#             return connection
#     except mysql.connector.Error as e:
#         print(f"Error while connecting to MySQL: {e}")
#     return None

# def export_to_csv(connection, filename='magic_cards.csv'):
#     query = """
#     SELECT name, colorIdentity, keywords, supertypes
#     FROM cards
#     """
#     df = pd.read_sql(query, connection)
#     df.to_csv(filename, index=False)
#     print(f"Data exported to {filename}")



In [233]:
# Main execution
# connection = connect_to_database()
# if connection:
#     export_to_csv(connection)
#     connection.close()
# else:
#     print("Failed to connect to the database.")

## Analysis portion

In [1]:
def load_card_data(filename='cards.csv'):
    return pd.read_csv(filename)

def prepare_features(df):
    # Process colors
    color_map = {'W': 'White', 'U': 'Blue', 'B': 'Black', 'R': 'Red', 'G': 'Green'}
    df['colorIdentity'] = df['colorIdentity'].fillna('').apply(lambda x: [color_map.get(c.strip(), c.strip()) for c in x.split(',')] if isinstance(x, str) else [])
    mlb_colors = MultiLabelBinarizer()
    color_features = pd.DataFrame(
        mlb_colors.fit_transform(df['colorIdentity']),
        columns=mlb_colors.classes_,
        index=df.index
    )
    
    # Process keywords
    df['keywords'] = df['keywords'].fillna('').apply(lambda x: [kw.strip() for kw in x.split(',')] if isinstance(x, str) else [])
    mlb_keywords = MultiLabelBinarizer()
    keyword_features = pd.DataFrame(
        mlb_keywords.fit_transform(df['keywords']),
        columns=mlb_keywords.classes_,
        index=df.index
    )
    
    # Process supertypes
    df['is_legendary'] = df['supertypes'].fillna('').str.contains('Legendary')
    
    # Combine features
    features = pd.concat([color_features, keyword_features, df[['is_legendary']]], axis=1)
    
    return features, mlb_colors, mlb_keywords

def cluster_cards(features, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=69, n_init=10)
    return kmeans.fit_predict(features)

def find_related_cards(df, features, legendary_card_name, top_n=1000):
    legendary_card = df[df['name'] == legendary_card_name].index[0]
    legendary_features = features.iloc[legendary_card].values.reshape(1, -1)
    
    similarities = cosine_similarity(legendary_features, features)
    similar_indices = similarities[0].argsort()[::-1][1:top_n+1]
    
    return df.iloc[similar_indices]

In [4]:
df_raw = load_card_data()
features, mlb_colors, mlb_keywords = prepare_features(df_raw)

/var/folders/zr/73hg6l9j7p739_l5lp2w946w0000gn/T/ipykernel_82229/3459460400.py:2: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filename)


In [ ]:
#cleans up data frame by removing unwanted columns 
def card_cleanup(df):
    #removes warning statement of fills
    pd.set_option('future.no_silent_downcasting', True)

    keep_params = ['colorIdentity', 'colors', 'edhrecRank',
                   'edhrecSaltiness', 'hasAlternativeDeckLimit',
                    'isReserved', 'keywords', 'layout',
                   'manaCost', 'manaValue', 'name', 'power',
                   'subtypes', 'supertypes', 'text', 'toughness',
                   'type', 'types', 'is_legendary',
                   'rarity', ]
    df_out = df[keep_params]
    #fixes Nans for strings
    df_out.loc[:, 'text'] = df_out['text'].fillna('').infer_objects(copy=False)

    #fixes Nans for boolean values
    df_out.loc[:, 'hasAlternativeDeckLimit'] = df_out['hasAlternativeDeckLimit'].fillna(False).infer_objects(copy=False) #assumes NA means it doesnt have a deck limit, aka false
    df_out.loc[:, 'isReserved'] = df_out['isReserved'].fillna(False).infer_objects(copy=False) #assumes NA means it isnt reserved
    
    #fixes nans for numeric
    #-1 symbolizes the data is not valid (ie, a sorcerydoesn't have toughness so it is set to -1)
    df_out.loc[:, 'edhrecRank'] = df_out['edhrecRank'].fillna(-1).infer_objects(copy=False)
    df_out.loc[:, 'edhrecSaltiness'] = df_out['edhrecSaltiness'].fillna(-1).infer_objects(copy=False)
    df_out.loc[:, 'toughness'] = df_out['toughness'].fillna(-1).infer_objects(copy=False)

    #others
    #special logic needed to set list to empty
    df_out.loc[:, 'subtypes'] = df_out['subtypes'].apply(lambda x: [] if pd.isna(x) else x)
    df_out.loc[:, 'supertypes'] = df_out['supertypes'].apply(lambda x: [] if pd.isna(x) else x)
    df_out.loc[:, 'colors'] = df_out['colors'].apply(lambda x: [] if pd.isna(x) else x)

    #should these be normalized or not even used?

    return df_out

In [19]:
df = card_cleanup(df_raw)

In [20]:
df[df['name'] == 'Kamiz, Obscura Oculus']

,colorIdentity,colors,edhrecRank,edhrecSaltiness,hasAlternativeDeckLimit,isReserved,keywords,layout,leadershipSkills,manaCost,...,name,power,subtypes,supertypes,text,toughness,type,types,is_legendary,rarity
54311,"[Black, Blue, White]","B, U, W",7232.0,0.25,False,False,[Connive],normal,"{""brawl"": false, ""commander"": true, ""oathbreak...",{1}{W}{U}{B},...,"Kamiz, Obscura Oculus",2,"Octopus, Rogue",Legendary,"Whenever you attack, target attacking creature...",4,Legendary Creature — Octopus Rogue,Creature,True,mythic
54412,"[Black, Blue, White]","B, U, W",7232.0,0.25,False,False,[Connive],normal,"{""brawl"": false, ""commander"": true, ""oathbreak...",{1}{W}{U}{B},...,"Kamiz, Obscura Oculus",2,"Octopus, Rogue",Legendary,"Whenever you attack, target attacking creature...",4,Legendary Creature — Octopus Rogue,Creature,True,mythic
54497,"[Black, Blue, White]","B, U, W",7232.0,0.25,False,False,[Connive],normal,"{""brawl"": false, ""commander"": true, ""oathbreak...",{1}{W}{U}{B},...,"Kamiz, Obscura Oculus",2,"Octopus, Rogue",Legendary,"Whenever you attack, target attacking creature...",4,Legendary Creature — Octopus Rogue,Creature,True,mythic


In [ ]:
# Perform clustering
clusters = cluster_cards(features)
df['cluster'] = clusters

/var/folders/zr/73hg6l9j7p739_l5lp2w946w0000gn/T/ipykernel_77812/3459460400.py:2: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filename)


In [9]:
df.columns

Index(['artist', 'artistIds', 'asciiName', 'attractionLights', 'availability',
       'boosterTypes', 'borderColor', 'cardParts', 'colorIdentity',
       'colorIndicator', 'colors', 'defense', 'duelDeck', 'edhrecRank',
       'edhrecSaltiness', 'faceConvertedManaCost', 'faceFlavorName',
       'faceManaValue', 'faceName', 'finishes', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized', 'isPromo',
       'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life', 'loyalty', 'manaCost',
       'manaValue', 'name', 'number', 'originalPrintings',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'rarity', 'rebalancedPrint

In [13]:
df.head()

,artist,artistIds,asciiName,attractionLights,availability,boosterTypes,borderColor,cardParts,colorIdentity,colorIndicator,...,supertypes,text,toughness,type,types,uuid,variations,watermark,is_legendary,cluster
0,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,[White],NaN,...,NaN,First strike (This creature deals combat damag...,4,Creature — Human Cleric,Creature,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,b7c19924-b4bf-56fc-aa73-f586e940bd42,NaN,False,7
1,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,[White],NaN,...,NaN,First strike (This creature deals combat damag...,4,Creature — Human Cleric,Creature,b7c19924-b4bf-56fc-aa73-f586e940bd42,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,NaN,False,7
2,Volkan Baǵa,93bec3c0-0260-4d31-8064-5d01efb4153f,NaN,NaN,"mtgo, paper",default,black,NaN,[White],NaN,...,NaN,"Flying\nWhen Angel of Mercy enters, you gain 3...",3,Creature — Angel,Creature,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a,NaN,False,7
3,Volkan Baǵa,93bec3c0-0260-4d31-8064-5d01efb4153f,NaN,NaN,"mtgo, paper",default,black,NaN,[White],NaN,...,NaN,"Flying\nWhen Angel of Mercy enters, you gain 3...",3,Creature — Angel,Creature,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,NaN,False,7
4,Mark Zug,48e2b98c-5467-4671-bd42-4c3746115117,NaN,NaN,"mtgo, paper",default,black,NaN,[White],NaN,...,NaN,Target creature gets +3/+3 and gains flying un...,NaN,Sorcery,Sorcery,55bd38ca-dc73-5c06-8f80-a6ddd2f44382,c5655330-5131-5f40-9d3e-0549d88c6e9e,NaN,False,1


In [236]:
print("Sample of clustered cards:")
print(df[['name', 'colorIdentity', 'keywords', 'is_legendary', 'cluster']].head(5))

Sample of clustered cards:
                name colorIdentity        keywords  is_legendary  cluster
0  Ancestor's Chosen       [White]  [First strike]         False        7
1  Ancestor's Chosen       [White]  [First strike]         False        7
2     Angel of Mercy       [White]        [Flying]         False        7
3     Angel of Mercy       [White]        [Flying]         False        7
4   Angelic Blessing       [White]              []         False        1


In [10]:
# Find related cards for a legendary creature
legendary_card_name = "Reya Dawnbringer"  # An example from your data
if legendary_card_name in df['name'].values:
    related_cards = find_related_cards(df, features, legendary_card_name)
    pd.set_option('display.max_rows', None)
    print(f"\nCards related to {legendary_card_name}:")
    print(related_cards[['name', 'colorIdentity', 'keywords', 'is_legendary', 'text']])
else:
    print(f"Legendary creature '{legendary_card_name}' not found in the database.")


Cards related to Reya Dawnbringer:
                                                    name   colorIdentity  \
13059                         Teshar, Ancestor's Apostle         [White]   
26166                                Zeriam, Golden Wind         [White]   
20525                         Teshar, Ancestor's Apostle         [White]   
76342                        Mavinda, Students' Advocate         [White]   
26242                                Zeriam, Golden Wind         [White]   
85668                        Mavinda, Students' Advocate         [White]   
79694                         Linvala, Keeper of Silence         [White]   
87725                                        Major Teroh         [White]   
76343                        Mavinda, Students' Advocate         [White]   
49                                      Reya Dawnbringer         [White]   
48                                      Reya Dawnbringer         [White]   
26677                                  Lieutenant Ki

In [238]:
# Print some statistics
print("\nCluster Statistics:")
print(df['cluster'].value_counts())

print("\nMost common keywords:")
print(df['keywords'].explode().value_counts().head(10))

print("\nColor distribution:")
print(df['colorIdentity'].explode().value_counts())


Cluster Statistics:
cluster
0    17077
6    15611
2    13611
4    10813
3    10324
1    10252
7     6593
8     6157
9     4250
5     3004
Name: count, dtype: int64

Most common keywords:
keywords
             57285
Flying        8811
Enchant       3294
Trample       2441
Haste         1784
Vigilance     1675
Equip         1463
Mill          1427
Flash         1384
Scry          1345
Name: count, dtype: int64

Color distribution:
colorIdentity
Green    22038
Black    21896
White    21690
Blue     21666
Red      21650
         10813
Name: count, dtype: int64


In [239]:
cards = pd.read_csv('cards.csv')
with open('Keywords.json') as json_file:
    keywords = json.load(json_file)

#load dict of all keywords
keywords = pd.DataFrame({'keywords': keywords})

#split rules text into all possible permutations, then do this for each candidate and compare amount of matching options to increase support
def split_rules(rules_text):
    splits = []
    #split at first occurrence and store, then second, and so on. then iterate and do it again
    parsed_rules = rules_text.split("\\n")[-1].split(" ")

    for i in range(len(parsed_rules)):
        cut_rules = " ".join(parsed_rules[:len(parsed_rules) - i])
        for j in range(len(cut_rules.split(" "))):
            #split at last occurrence, then take the first value in split to get full string
            sliced_rules = cut_rules.split("\\n")[-1].split(" ", j)
            # for j in range(len(bruh) - i):
            splits.append(sliced_rules[-1])
    return splits

def create_candidates(ref_name, related_cards):
    # print(ref_name)
    candidates = []

    rules_matrix = []

    for text in related_cards['text'].tolist():
        rules_matrix.append(split_rules(text))
        
    for i in range(len(rules_matrix)):
        if len(set(rules_matrix[0]) & set(rules_matrix[i])) > 2 and len(candidates) < 62 and (related_cards['name'].tolist()[i], rules_matrix[i][0], len(set(rules_matrix[0]) & set(rules_matrix[i]))) not in candidates:
            #currently only preserving name during testing
            candidates.append((related_cards['name'].tolist()[i], rules_matrix[i][0], len(set(rules_matrix[0]) & set(rules_matrix[i]))))
            #candidates += create_candidates(related_cards['name'].tolist()[i], related_cards)

    return candidates


cell_value = cards.where(cards=='Reya Dawnbringer').dropna(how='all').dropna(axis=1)
# print(related_cards[['name', 'text']])
#increase support value based on shared keywords, shared terms in rules text, and triggers
pd.set_option('display.max_colwidth', None)
#print(cards.loc[cell_value.index[-1], 'text']) 
# rules = split_rules(cards.loc[cell_value.index[-1], 'text'])

candidates = create_candidates(legendary_card_name, related_cards)
print(len(candidates))

C:\Users\Jackie\AppData\Local\Temp\ipykernel_16180\2660854644.py:1: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  cards = pd.read_csv('cards.csv')


('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your opponents can't cast spells of the chosen color.", 45)
('Iona, Shield of Emeria', "Your

KeyboardInterrupt: 